In [1]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [3]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [4]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [5]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [6]:
data.shape

(988, 5)

In [7]:
data.head(3)

,content,keywords,summary,title,url
0,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","[школа, образовательные стандарты, литература, история, фгос]","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html
1,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...","[красота, законы]",О живительной пользе укорота при выборе между плохим и хорошим,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html
2,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...","[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [8]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [9]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [10]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [11]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.05
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [12]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [13]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.0
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [14]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                      [ольга, васильева, обещала, "нг", не, перегружать, школьников]
1                                           [у, красоты, собственные, закон, и, воля]
2                                                            [апокалиптический, бунт]
3                   [f1., предсказать, результаты, гран-при, испании, было, несложно]
4                                               [возвращение, в, небесное, отечество]
5                                       [практическая, медицина, с, большим, будущим]
6                     [бумажный, носитель., вселенная,, человек,, язык, человеческий]
7                                      [в, багдаде, неспокойно,, а, будет, еще, хуже]
8    [ученые, предсказывают, появление, интуиции, у, искусственных, нейронных, сетей]
9                                  [россияне, от, вступления, в, вто, не, пострадают]
Name: title, dtype: object

In [15]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                            [и, в, –, по, на, что, с, о, к, не]
1                 [и, в, не, что, –, на, красота, вы, если, как]
2                     [в, и, не, на, –, от, о, из, пепеляев, но]
3                        [в, не, и, на, что, но, это, как, с, к]
4                          [и, в, –, не, он, с, на, что, а, его]
5                       [и, в, на, –, с, что, не, это, как, уже]
6                           [–, и, в, не, на, из, по, что, с, а]
7                       [в, и, не, на, но, ли, ираке, что, -, а]
8                           [и, в, на, о, с, –, как, а, что, ai]
9    [в, на, по, и, что, кристалина, –, россии, георгиева, том,]
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [16]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [17]:
data['content_norm'] = data['content'].apply(normalize)

In [18]:
data['title_norm'] = data['title'].apply(normalize)

In [19]:
data['title_norm'].head(10)

0                           [ольга, васильев, обещать, нг, перегружать, школьник]
1                                             [красота, собственный, закон, воля]
2                                                        [апокалиптический, бунт]
3                       [f1, предсказать, результат, гран-при, испания, несложно]
4                                              [возвращение, небесный, отечество]
5                                      [практический, медицина, больший, будущий]
6                    [бумажный, носитель, вселенная, человек, язык, человеческий]
7                                                     [багдад, неспокойно, худой]
8    [учёный, предсказывать, появление, интуиция, искусственный, нейронный, сеть]
9                                        [россиянин, вступление, вто, пострадать]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [20]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [21]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.12
F1 -  0.11
Jaccard -  0.07


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [22]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0     [стандарт, который, источник, образовательный, фгоса, ольга, васильев, результат, предмет, исторический]
1                                          [красота, ваш, глаз, отчаяние, уйти, это, порыв, кошка, свой, один]
2                               [пепеляев, юзеф, книга, якутия, восстание, год, самый, война, место, когда-то]
3                              [гонка, команда, это, сказать, пилот, ferrari, mclaren, сезон, mercedes, время]
4                                          [есенин, поэт, клюев, год, свой, это, жизнь, который, смерть, 1925]
5                        [наш, медицина, медицинский, это, кафедра, выпускник, работать, уровень, работа, год]
6                               [книга, русский, человек, мозг, два, островной, говор, это, анатомия, который]
7                     [ирак, война, американец, это, партизанский, войско, структура, единый, сказать, важный]
8                       [который, нейросеть, свой, ai, клетка, искусственный, интеллект, самый, время, основа]
9

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [89]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [90]:
data['content_norm'] = data['content'].apply(normalize)

In [88]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


Ещу улучшения!

In [27]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0    [стандарт, источник, фгоса, ольга, васильев, результат, предмет, школа, письмо, произведение]
1                      [красота, глаз, отчаяние, порыв, кошка, предмет, дело, руина, место, мечта]
2                    [пепеляев, юзеф, книга, якутия, восстание, год, война, место, леонид, сибирь]
3                    [гонка, команда, пилот, сезон, время, машина, место, круг, гран-при, испания]
4                    [есенин, поэт, клюев, год, жизнь, смерть, сергей, человек, борода, мариенгоф]
5       [медицина, кафедра, выпускник, уровень, работа, год, практика, ординатор, рудна, обучение]
6               [книга, человек, мозг, говор, анатомия, глава, звезда, вопрос, слово, азербайджан]
7       [ирак, война, американец, войско, структура, принцип, взрыв, центр, террорист, количество]
8    [нейросеть, клетка, интеллект, время, основа, изображение, задача, работа, система, внимание]
9      [россия, вступление, кристалина, георгиев, вто, банка, страна, переговоры, директор, слово]
Name: cont

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [48]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [49]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [50]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [57]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [52]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [53]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [55]:
keywords[:3]

[['ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'письмо',
  'произведение',
  'редакция',
  'содержание'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'предмет',
  'мечта',
  'безобразие',
  'познание',
  'англичанин'],
 ['якутия',
  'книга',
  'восстание',
  'сибирь',
  'леонид',
  'красных',
  'пустынь',
  'перо',
  'война',
  'мороз']]

In [56]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [59]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [60]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [61]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 45.9 s, sys: 128 ms, total: 46.1 s
Wall time: 1min 15s


In [62]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [67]:
keywords_rw.head(10)

0           [стандарт, образование, источник, произведение, обучение, рф, письмо, класс, предмет, совет]
1                       [красота, отчаяние, порыв, глаз, кошка, место, пение, предмет, обаяние, плюхома]
2                               [юзеф, якутия, поэт, место, сибирь, восстание, посёлок, год, иван, пуля]
3                   [команда, гонка, пилот, машина, хэмилтон, круг, внимание, борьба, дженсон, гран-при]
4                           [есенин, поэт, год, борода, лев, конец, пружина, жизнь, сереженька, человек]
5           [кафедра, медицина, уровень, выпускник, ординатор, врач, обучение, работа, знание, практика]
6                      [книга, человек, тираж, говор, наука, трус, азербайджан, работа, этинген, шутник]
7    [ирак, американец, война, деятельность, территория, взрывчатка, полгода, почерк, гражданин, багдад]
8         [нейросеть, интеллект, клетка, изображение, система, метод, время, внимание, пиксель, решение]
9                [россия, георгиев, банка, вступление, 

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [68]:
import networkx as nx

In [69]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.eigenvector_centrality_numpy(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [66]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))
print(evaluate(data['keywords'], keyword_nx))

Precision -  0.18
Recall -  0.17
F1 -  0.16
Jaccard -  0.1
None
CPU times: user 12 s, sys: 37.3 ms, total: 12 s
Wall time: 19.2 s


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [70]:
from gensim.summarization import keywords

In [71]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [72]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


Наша имплементация отработала получше.

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/H9lBH9wCxqq1T0ru2

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

## Эксперимент 1 (удачный). LSI

In [47]:
import gensim
dct = gensim.corpora.dictionary.Dictionary(data['content_norm'])
corpus = [dct.doc2bow(line) for line in data['content_norm']]
tdidf_model = gensim.models.TfidfModel(corpus)
tfidf_corpus = tdidf_model[corpus]
lsi_model = gensim.models.LsiModel(tfidf_corpus, num_topics=500)

In [73]:
%%time
from sklearn.metrics.pairwise import cosine_similarity

def vectorize(tokens):
    return lsi_model[dct.doc2bow(tokens)]

def get_keywords_with_scores(tokens):
    corpus = vectorize(tokens)
    sim = gensim.similarities.MatrixSimilarity([corpus])
    result = list()
    for token in tokens:
        similarity = sim.get_similarities(vectorize([token]))[0]
        result.append((token, similarity))
    
    result = sorted(result, key = lambda x: x[1], reverse = True)
    return result

def get_best_keywords(tokens, n):
    keywords_with_scores = get_keywords_with_scores(tokens)
    return [word for word, score in keywords_with_scores][:n]

keywords_lsi = [get_best_keywords(sent, 30) for sent in data['content_norm']]

/home/vadim/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


CPU times: user 2min 50s, sys: 392 ms, total: 2min 51s
Wall time: 3min 27s


In [75]:
evaluate(data['keywords'], keywords_lsi)

Precision -  0.24
Recall -  0.21
F1 -  0.18
Jaccard -  0.11


## Эксперимент 2 (удачный). Графы

In [584]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.eigenvector_centrality_numpy(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [585]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 5))
print(evaluate(data['keywords'], keyword_nx))

Precision -  0.18
Recall -  0.17
F1 -  0.17
Jaccard -  0.1
None
CPU times: user 12.5 s, sys: 75.5 ms, total: 12.6 s
Wall time: 17.5 s


## Эксперимент 3 (неудачный). rutermextract

In [651]:
import rutermextract

In [656]:
from rutermextract import TermExtractor
term_extractor = TermExtractor()

2019-01-29 05:28:11,180 : INFO : Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts/data
2019-01-29 05:28:11,247 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [663]:
%%time
keywords_ruterm = [[term.normalized for term in term_extractor(sent)] for sent in data['content_norm_str']]

CPU times: user 1min 6s, sys: 125 ms, total: 1min 7s
Wall time: 1min 17s


In [675]:
evaluate(data['keywords'], [x[:2] for x in keywords_ruterm])

Precision -  0.13
Recall -  0.22
F1 -  0.15
Jaccard -  0.09


## Эксперимент 4 (неудачный). Ансамблинг

In [681]:
keywords_ensamble = [list(set(kw1) & set(kw2)) for kw1, kw2 in zip(keywords_ruterm, keyword_nx)]

In [682]:
evaluate(data['keywords'], keywords_ensamble)

Precision -  0.17
Recall -  0.16
F1 -  0.16
Jaccard -  0.09


## Эксперимент 5 (неудачный). Эмбеддинги

In [78]:
model = gensim.models.KeyedVectors.load_word2vec_format('model/ru1/ru.vec', binary=False, unicode_errors='ignore')

In [84]:
def get_idf(token):
    return tfidf.idf_[tfidf.vocabulary_[token]]

In [85]:
def vectorize_sentence(sentence):
    result = list()
    for token in sentence:
        try:
            result.append(model[token] * get_idf(token))
        except KeyError:
            pass
    
    if len(result) > 0:
        return np.mean(result, 0)
    else:
        raise KeyError("Sentence {} does not contain any words that I know".format(sentence))
    
def keyword_lookup(sentence):
    result = list()
    sentence_vector = vectorize_sentence(sentence)
    candidates = set(sentence + [word for word, score in model.similar_by_vector(sentence_vector)])
    for candidate in candidates:
        if candidate in model:
            if not candidate in [word for word, score in result]:
                similarity = cosine_similarity([sentence_vector], [model[candidate]])[0][0]
                result.append((candidate, similarity))
    
    result = sorted(result, key = lambda x: x[1], reverse = True)
    
    return result

def get_best_keywords(sentence, n):
    return [word for word, score in keyword_lookup(sentence)][:n]

In [86]:
keywords_word2vec = [get_best_keywords(sent, 10) for sent in data['content_norm']]

In [87]:
evaluate(data['keywords'], keywords_word2vec)

Precision -  0.05
Recall -  0.09
F1 -  0.06
Jaccard -  0.03
